In [ ]:
import sys

# install mirp
!{sys.executable} -m pip install --upgrade -q mirp

# install tcia utils for downloading example data
!{sys.executable} -m pip install --upgrade -q tcia_utils

In [ ]:
from mirp import extract_mask_labels
from mirp import extract_image_parameters
from mirp import extract_images
from mirp import extract_features
import pandas as pd
from tcia_utils import nbia

# Download example data

In this example we will use the [Soft-tissue-Sarcoma](http://doi.org/10.7937/K9/TCIA.2015.7GO2GSKS) dataset from The Cancer Imaging Archive.  Please be sure to abide by the [TCIA Data Usage Policy and Restrictions](https://www.cancerimagingarchive.net/data-usage-policies-and-restrictions/) by including this citation anywhere these data are being discussed:

```Vallières, Martin, Freeman, Carolyn R., Skamene, Sonia R., & El Naqa, Issam. (2015). A radiomics model from joint FDG-PET and MRI texture features for the prediction of lung metastases in soft-tissue sarcomas of the extremities (Soft-tissue-Sarcoma) [Dataset]. The Cancer Imaging Archive. http://doi.org/10.7937/K9/TCIA.2015.7GO2GSKS```

In order to download the data we'll utilize [tcia_utils](https://pypi.org/project/tcia-utils/).  These commands will demonstrate how to create a dataframe of image and segmentation metadata that we can use to execute mirp commands, as well as how to download the segmentations (RTSTRUCT) and reference MRIs to a folder called `tciaDownload/< SeriesInstanceUID`.  

In [ ]:
# download an inventory of all scans in the Soft-tissue-Sarcoma dataset
series_metadata = nbia.getSeries(collection = "Soft-tissue-Sarcoma", format = "df")

# sort it display the first subject's scans
series_metadata = series_metadata.sort_values(by=['PatientID', 'StudyInstanceUID', 'SeriesNumber'], ascending=[True, True, True])
series_metadata[series_metadata['PatientID'].isin(series_metadata['PatientID'].unique()[:1])]

After reviewing the data, let's say we want to focus on the T1 segmentations.  First we'll extract the SeriesInstanceUIDs and other key information to a new dataframe.

In [ ]:
# Copy only the "RTstruct_T1" segmentation metadata to a new dataframe
seg_metadata = series_metadata[series_metadata['SeriesDescription'] == "RTstruct_T1"].copy().reset_index(drop=True)
seg_metadata = seg_metadata[['PatientID', 'StudyInstanceUID', 'SeriesInstanceUID', 'SeriesDescription', 'Modality', 'BodyPartExamined']]

seg_metadata

Next we'll add a ReferencedSeriesInstanceUID column and use the TCIA API to look up the SeriesInstanceUID of the original T1 scans that were segmented.

In [ ]:
# Create an empty list to store the referenced series instance UIDs
referenced_series_uids = []

# Iterate through each row in the DataFrame and fetch the referenced series instance UID
for index, row in seg_metadata.iterrows():
    series_instance_uid = row['SeriesInstanceUID']
    referenced_series_uid = nbia.getSegRefSeries(series_instance_uid)
    referenced_series_uids.append(referenced_series_uid)

# Add the list of referenced series instance UIDs as a new column in the DataFrame
seg_metadata['ReferencedSeriesInstanceUID'] = referenced_series_uids

Now we can download some images.  We'll start with 3 subjects, but you can change or remove the number parameter if you like.

In [ ]:
# set the number of patients or use None to download the full set of scans
num_patients = 3

if num_patients is None:
    uids_list = list(seg_metadata['SeriesInstanceUID']) + list(seg_metadata['ReferencedSeriesInstanceUID'])
else:
    selected_patients = seg_metadata.head(num_patients)
    uids_list = list(selected_patients['SeriesInstanceUID']) + list(selected_patients['ReferencedSeriesInstanceUID'])

# Pass the uids_list to nbia.downloadSeries(uids) for downloading
nbia.downloadSeries(uids_list, input_type = "list")


In [ ]:
selected_patients

# Finding mask labels

Radiomics features are typically computed from regions of interest, such as a tumour. These regions are delineated by experts or auto-segmentation AI, and stored as segmentation masks. MIRP needs to know which mask label (region of interest) should be used for computing features. A first step is to identify which mask labels exist. This can be done using the extract_mask_labels function. We can use our previously created dataframe to tell MIRP where the masks can be found.

In [ ]:
# Create an empty DataFrame to store the concatenated results
all_labels = pd.DataFrame()

# Loop through the selected_patients DataFrame
for index, row in selected_patients.iterrows():
    # Set seg_path for each row
    seg_path = "tciaDownload/" + row['SeriesInstanceUID'] + "/1-1.dcm"
    # Set labels by calling extract_mask_labels(mask = seg_path)
    labels = extract_mask_labels(mask=seg_path)
    
    # Concatenate the results into the all_labels DataFrame
    all_labels = pd.concat([all_labels, labels])
    
all_labels

We are lucky that all masks are consistently labelled. GTV_Mass and GTV_Edema both refer to the gross tumour volume, i.e. that part of the tumour that is visible in medical imaging. GTV-Edema also covers fluid surrounding the gross tumour volume itself.

# Visualising images
It is often useful to inspect images before computing radiomics features. External viewers for DICOM and many other image types exist, but MIRP also has a simple visualisation tool. You can visualise images by exporting them in MIRP internal formats using extract_images:

In [ ]:
selected_patient

In [ ]:
# choose a patient to visualize
patient_id = "STS_003"

# filter the dataframe to that patient
selected_patient = selected_patients[selected_patients['PatientID'] == patient_id]

# Set seg_path and image_path
for index, row in selected_patient.iterrows():
    seg_path = "tciaDownload/" + row['SeriesInstanceUID'] + "/1-1.dcm"
    image_path = "tciaDownload/" + row['ReferencedSeriesInstanceUID']

# extract parameters
images = extract_images(image = image_path, 
                            mask = seg_path,
                            roi_name = "GTV_Mass",
                            image_export_format = "native")

In [ ]:
image, mask = images[0]
image[0].show(mask=mask[0], slice_id=10)

There's also a tcia_utils function for this you might want to try.  It'd be great if we could collaborate on this in the future so we're not both maintaining something to do this.  I'd much rather not be in the business of doing visualization if you're interested in rolling this function into your tool and improving it (there are issues with some image modalities and segmentation types) but I couldn't find anything super simple to do what we're both trying to achieve here so I had a summer intern help put this together last year.

In [ ]:
nbia.viewSeriesAnnotation(seriesPath = image_path, annotationPath = seg_path)

# Assessing image metadata
Image metadata are important for understanding the image and how it was acquired and reconstructed. MIRP allows for exporting image metadata from DICOM and other image formats, though for non-DICOM formats metadata will be considerably more limited.

In [ ]:
# Create an empty DataFrame to store the concatenated results
all_parameters = pd.DataFrame()

# Loop through the selected_patients DataFrame
for index, row in selected_patients.iterrows():
    # Set seg_path for each row
    image_path = "tciaDownload/" + row['ReferencedSeriesInstanceUID']
    # Extract parameters for each image series
    parameters = extract_image_parameters(image = image_path)
    
    # Concatenate the results into the all_labels DataFrame
    all_parameters = pd.concat([all_parameters, parameters])
    
all_parameters

Only known metadata are shown. For example, magnetic field strength was not present in the image metadata in this example.

The metadata have important implications for the image processing:

* The in-plane resolution is much higher than the distance between slices. This suggests that features should be computed by slice, instead in 3D.
* The in-plane resolution differs between patients. This suggests that the images should be resampled to isotropic pixel sizes, e.g. 1.0 by 1.0 mm.
* All three images were recorded in different scanners. This suggests that MR intensities cannot be compared between patients, and should be standardised.

# Computing features
The presented metadata suggest that image processing is required to make the MR images more comparable between patients. We will define three image processing steps:

1. Image processing and feature computation is performed by slice (by_slice=True) due to large distances between image slices.
2. In-plane resolution is resampled to 1.0 by 1.0 mm (new_spacing=1.0).
3. Intensities are normalised, here using z-normalisation (intensity_normalisation="standardisation").

In addition, we need to define parameters related to intensity discretisation for computing histogram-based and texture features. Since intensities were normalised using z-normalisation, we will use a fixed bin number algorithm `(base_discretisation_method="fixed_bin_number")` with 16 bins `(base_discretisation_n_bins=16)`.

In [ ]:
# Create an empty list to store the DataFrames
all_features_list = []

# Loop through the selected_patients DataFrame
for index, row in selected_patients.iterrows():
    # Set image_path and seg_path for each row
    image_path = "tciaDownload/" + row['ReferencedSeriesInstanceUID']
    seg_path = "tciaDownload/" + row['SeriesInstanceUID'] + "/1-1.dcm"
    
    # Extract parameters for each image series
    features = extract_features(
        image=image_path,
        mask=seg_path,
        roi_name="GTV_Mass",
        by_slice=True,
        intensity_normalisation="standardisation",
        new_spacing=1.0,
        base_discretisation_method="fixed_bin_number",
        base_discretisation_n_bins=16
    )
    
    # Extend the list of DataFrames
    all_features_list.extend(features)

# Concatenate all DataFrames in the list
all_features = pd.concat(all_features_list, ignore_index=True)
all_features


This results in a pandas.DataFrame that has a row per image and mask. The first several columns contain parameters related to that image and mask, and how these were processed. After these, feature values are shown. These can be used for, e.g., machine learning using `scikit-learn <https://scikit-learn.org/stable/>`__ or `familiar <https://cran.r-project.org/web/packages/familiar/index.html>`__.